# Import Library

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Import Dataset

In [8]:
#path data
df_idx = '../dataset/raw/historical_data_idx30.csv'
df_kurs = '../dataset/raw/historical_data_kurs.csv'
df_nyse = '../dataset/raw/historical_data_nyse.csv'
df_sb = '../dataset/raw/data_suku_bunga.xlsx'

#buat dataframe
df_idx = pd.read_csv(df_idx)
df_kurs = pd.read_csv(df_kurs, sep=';')
df_nyse = pd.read_csv(df_nyse, sep=',')
df_sb = pd.read_excel(df_sb)

# Dataset Information

## IDX30

In [161]:
df_idx.info() #cek  informasi dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1422 entries, 0 to 1421
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Tanggal     1422 non-null   object
 1   Terakhir    1422 non-null   object
 2   Pembukaan   1422 non-null   object
 3   Tertinggi   1422 non-null   object
 4   Terendah    1422 non-null   object
 5   Vol.        1418 non-null   object
 6   Perubahan%  1422 non-null   object
dtypes: object(7)
memory usage: 77.9+ KB


In [162]:
df_idx.duplicated().sum() #cek jumlah data terduplikasi

0

In [163]:
df_idx.isna().sum() #cek data yang memiliki null value

Tanggal       0
Terakhir      0
Pembukaan     0
Tertinggi     0
Terendah      0
Vol.          4
Perubahan%    0
dtype: int64

In [164]:
df_idx[df_idx.isna().any(axis=1)] #menampilkan sebaran data yang memiliki null value

,Tanggal,Terakhir,Pembukaan,Tertinggi,Terendah,Vol.,Perubahan%
372,01/05/2024,"470,64","468,11","474,52","466,67",NaN,"0,00%"
450,26/12/2023,"491,41","491,85","493,10","489,99",NaN,"0,00%"
850,11/05/2022,"549,38","549,10","557,39","547,74",NaN,"0,34%"
956,29/11/2021,"508,40","501,86","510,29","495,52",NaN,"1,11%"


In [165]:
df_idx['Tanggal'] = pd.to_datetime(df_idx['Tanggal'], format='%d/%m/%Y') #mengubah tipe data tanggal menjadi datetime
df_idx['Perubahan%'] = pd.to_numeric(df_idx['Perubahan%'].astype(str).str.replace('%', '').str.replace(',','.')) #merubah 'perubahan%' menjadi data numerik

change_type = ['Terakhir', 'Pembukaan', 'Tertinggi', 'Terendah'] #deretan kolom yang akan diubah tipe datanya

for i in change_type:
    df_idx[i] = df_idx[i].str.replace(',', '.', regex=False).astype(float) #mengganti koma menjadi titik
    df_idx[i] = pd.to_numeric(df_idx[i]) #mengubah tipe data menjadi numerik
    print(f"kolom {i} telah diubah menjadi numeric")

kolom Terakhir telah diubah menjadi numeric
kolom Pembukaan telah diubah menjadi numeric
kolom Tertinggi telah diubah menjadi numeric
kolom Terendah telah diubah menjadi numeric


In [166]:
df_idx['Vol.'].str[-1].value_counts() #melihat satuan bilangan dalam volume dalam kategorikal

Vol.
B    1284
M     134
Name: count, dtype: int64

In [167]:
def convert_volume(volume):
    if pd.isna(volume):
        return np.nan #kembalikan nilai kosong menjadi NaN

    volume = str(volume) #ubah value data menjadi string
    
    if volume.endswith('B'):
        return float(volume.replace(',', '.').replace('B','')) * 1000000000 #menkonversi data yang memiliki akhiran B
    else:
        return float(volume.replace(',', '.').replace('M','')) * 1000000

df_idx['Vol.'] = df_idx['Vol.'].apply(convert_volume)

In [168]:
df_idx.info() #melihat informasi dataset setelah perubahan tipe data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1422 entries, 0 to 1421
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Tanggal     1422 non-null   datetime64[ns]
 1   Terakhir    1422 non-null   float64       
 2   Pembukaan   1422 non-null   float64       
 3   Tertinggi   1422 non-null   float64       
 4   Terendah    1422 non-null   float64       
 5   Vol.        1418 non-null   float64       
 6   Perubahan%  1422 non-null   float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 77.9 KB


In [169]:
df_idx.head(5) #melihat sebaran data dalam dataset

,Tanggal,Terakhir,Pembukaan,Tertinggi,Terendah,Vol.,Perubahan%
0,2025-11-19,442.67,441.04,443.58,440.28,4.810000e+09,0.55
1,2025-11-18,440.24,444.51,446.18,438.73,5.140000e+09,-1.00
2,2025-11-17,444.68,444.02,446.76,443.66,6.440000e+09,0.35
3,2025-11-14,443.13,441.46,445.12,440.84,3.950000e+09,0.43
4,2025-11-13,441.25,446.11,446.52,440.89,6.540000e+09,-1.09


In [170]:
drop_columns = ['Pembukaan', 'Tertinggi', 'Terendah', 'Vol.', 'Perubahan%'] #kolom yang akan dihapus

for i in drop_columns:
    df_idx = df_idx.drop(columns=i) #menghapus kolom
    print(f"kolom {i} telah dihapus")

kolom Pembukaan telah dihapus
kolom Tertinggi telah dihapus
kolom Terendah telah dihapus
kolom Vol. telah dihapus
kolom Perubahan% telah dihapus


In [171]:
change_name = {'Tanggal' : 'date', 
                'Terakhir' : 'priceidx'}

df_idx = df_idx.rename(columns=change_name) #merubah nama kolom

for i, n in change_name.items():
    print(f"nama kolom {i} telah diubah menjadi {n}")

nama kolom Tanggal telah diubah menjadi date
nama kolom Terakhir telah diubah menjadi priceidx


In [172]:
df_idx.info() #melihat informasi data setelah perubahan nama kolom

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1422 entries, 0 to 1421
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      1422 non-null   datetime64[ns]
 1   priceidx  1422 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 22.3 KB


In [173]:
df_idx.describe() #melihat statistik deskriptif setelah perubahan

,date,priceidx
count,1422,1422.000000
mean,2022-12-05 12:02:01.518987520,473.653502
min,2020-01-02 00:00:00,311.880000
25%,2021-06-22 06:00:00,440.350000
50%,2022-11-28 12:00:00,484.060000
75%,2024-05-29 18:00:00,504.425000
max,2025-11-19 00:00:00,583.110000
std,NaN,47.198739


## NYSE

In [16]:
df_nyse.sort_values(by='Date',ascending=True).head(10) #melihat sebaran data

,Date,Price,Open,High,Low,Vol.,Change %
1479,01/02/2020,"14,002.5","13,913.0","14,003.4","13,913.0",NaN,0.64%
473,01/02/2024,"16,841.6","16,852.9","16,894.5","16,757.8",NaN,-0.07%
221,01/02/2025,"19,095.4","19,097.1","19,254.7","19,014.4",NaN,-0.01%
1478,01/03/2020,"13,917.0","13,877.5","13,950.7","13,870.7",NaN,-0.61%
974,01/03/2022,"17,226.1","17,164.1","17,231.0","17,120.0",NaN,0.36%
723,01/03/2023,"15,154.5","15,184.3","15,327.5","15,053.2",NaN,-0.20%
472,01/03/2024,"16,718.7","16,841.6","16,841.6","16,690.5",NaN,-0.73%
220,01/03/2025,"19,254.3","19,095.4","19,276.7","19,095.4",NaN,0.83%
1226,01/04/2021,"14,376.7","14,524.8","14,605.8","14,258.2",NaN,-1.02%
973,01/04/2022,"17,336.8","17,226.1","17,377.4","17,226.1",NaN,0.64%


In [13]:
df_nyse.info() #melihat informasi dataset setelah perubahan kolom

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1480 entries, 0 to 1479
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Date      1480 non-null   object 
 1   Price     1480 non-null   object 
 2   Open      1480 non-null   object 
 3   High      1480 non-null   object 
 4   Low       1480 non-null   object 
 5   Vol.      0 non-null      float64
 6   Change %  1480 non-null   object 
dtypes: float64(1), object(6)
memory usage: 81.1+ KB


In [14]:
df_nyse.duplicated().sum() #cek jumlah data yang terduplikasi

0

In [15]:
df_nyse.isna().sum() #cek jumlah data yang memiliki null value

Date           0
Price          0
Open           0
High           0
Low            0
Vol.        1480
Change %       0
dtype: int64

In [180]:
df_nyse['Price'] = pd.to_datetime(df_nyse['Price'], format='%Y-%m-%d') #mengubah data 'price' menjadi datetime

column_change = ['Close', 'High', 'Low', 'Open', 'Volume'] #kolom yang akan mengubah tipe data

for i in column_change:
    df_nyse[i] = pd.to_numeric(df_nyse[i]) #ubah tipe data
    print(f"kolom {i} telah diubah menjadi numerik")

kolom Close telah diubah menjadi numerik
kolom High telah diubah menjadi numerik
kolom Low telah diubah menjadi numerik
kolom Open telah diubah menjadi numerik
kolom Volume telah diubah menjadi numerik


In [181]:
df_nyse.info() #melihat informasi dataset setelah perubahan tipe data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1479 entries, 0 to 1478
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Price   1479 non-null   datetime64[ns]
 1   Close   1479 non-null   float64       
 2   High    1479 non-null   float64       
 3   Low     1479 non-null   float64       
 4   Open    1479 non-null   float64       
 5   Volume  1479 non-null   int64         
dtypes: datetime64[ns](1), float64(4), int64(1)
memory usage: 69.5 KB


In [182]:
df_nyse.describe() #melihat statistik deskriptif data

,Price,Close,High,Low,Open,Volume
count,1479,1479.000000,1479.000000,1479.000000,1479.000000,1.479000e+03
mean,2022-12-09 14:01:13.022312448,16366.760628,16460.150110,16263.556352,16363.926795,4.499201e+09
min,2020-01-02 00:00:00,8777.379883,9053.490234,8664.940430,9014.580078,1.639500e+09
25%,2021-06-21 12:00:00,14964.299805,15061.450195,14858.079590,14960.589844,3.814880e+09
50%,2022-12-07 00:00:00,16156.509766,16273.750000,16062.040039,16152.120117,4.266290e+09
75%,2024-05-29 12:00:00,18030.419922,18129.044922,17948.679688,18028.320312,4.911135e+09
max,2025-11-18 00:00:00,21807.330078,21876.990234,21718.509766,21807.330078,9.976520e+09
std,NaN,2536.815411,2528.223936,2541.772247,2531.107382,1.063057e+09


In [183]:
drop_columns = ['High', 'Low', 'Open', 'Volume'] #kolom yang akan dihapus

for i in drop_columns:
    df_nyse = df_nyse.drop(columns=i) #hapus kolom
    print(f"kolom {i} telah dihapus")

kolom High telah dihapus
kolom Low telah dihapus
kolom Open telah dihapus
kolom Volume telah dihapus


In [184]:
change_name = {
    'Price' : 'date',
    'Close' : 'pricenyse'
}

df_nyse = df_nyse.rename(columns=change_name) #mengubah nama kolom

for i, j in change_name.items():
    print(f"kolom {i} telah berubah nama menjadi {j}")

kolom Price telah berubah nama menjadi date
kolom Close telah berubah nama menjadi pricenyse


In [185]:
df_nyse.head() #melihat sebaran data setelah perubahan nama kolom

,date,pricenyse
0,2020-01-02,14002.490234
1,2020-01-03,13917.049805
2,2020-01-06,13941.799805
3,2020-01-07,13898.450195
4,2020-01-08,13934.440430


## Kurs

In [186]:
df_kurs.head() #melihat data dalam bentuk dataframe

,Date,Price,Open,High,Low,Vol.,Change %
0,11/19/2025,"16,716.6","16,711.6","16,760.2","16,680.8",NaN,-0.07
1,11/18/2025,"16,728.3","16,711.6","16,772.1","16,718.3",NaN,0.02
2,11/17/2025,"16,725.0","16,712.5","16,738.0","16,710.0",NaN,0.19
3,11/14/2025,"16,694.0","16,705.0","16,717.5","16,694.0",NaN,-0.19
4,11/13/2025,"16,725.0","16,725.0","16,738.5","16,717.5",NaN,0.17


In [187]:
df_kurs.info() #melihat informasi dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1481 entries, 0 to 1480
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Date      1481 non-null   object 
 1   Price     1481 non-null   object 
 2   Open      1481 non-null   object 
 3   High      1481 non-null   object 
 4   Low       1481 non-null   object 
 5   Vol.      744 non-null    object 
 6   Change %  1481 non-null   float64
dtypes: float64(1), object(6)
memory usage: 81.1+ KB


In [188]:
df_kurs.duplicated().sum() #cek jumlah data yang terduplikasi

0

In [189]:
df_kurs.isna().sum() #cek jumlah data yang memiliki null value

Date          0
Price         0
Open          0
High          0
Low           0
Vol.        737
Change %      0
dtype: int64

In [190]:
df_kurs[df_kurs.isna().any(axis=1)] #menampilkan data yang memiliki null value

,Date,Price,Open,High,Low,Vol.,Change %
0,11/19/2025,"16,716.6","16,711.6","16,760.2","16,680.8",NaN,-0.07
1,11/18/2025,"16,728.3","16,711.6","16,772.1","16,718.3",NaN,0.02
2,11/17/2025,"16,725.0","16,712.5","16,738.0","16,710.0",NaN,0.19
3,11/14/2025,"16,694.0","16,705.0","16,717.5","16,694.0",NaN,-0.19
4,11/13/2025,"16,725.0","16,725.0","16,738.5","16,717.5",NaN,0.17
...,...,...,...,...,...,...,...
732,01/16/2023,"15,040.0","15,080.0","15,080.0","14,981.5",NaN,-0.66
733,01/13/2023,"15,140.0","15,210.0","15,237.5","15,135.5",NaN,-1.28
734,01/12/2023,"15,336.0","15,470.0","15,470.0","15,290.0",NaN,-0.93
1237,01/01/2021,"14,213.7","14,213.5","14,214.2","13,919.0",NaN,1.24


In [191]:
drop_columns = ['Open', 'High', 'Low',  'Vol.', 'Change %'] #kolom data yang akan dihapus

for i in drop_columns:
    df_kurs = df_kurs.drop(columns=i) #hapus kolom dalam data
    print(f"kolom {i} telah dihapus")

kolom Open telah dihapus
kolom High telah dihapus
kolom Low telah dihapus
kolom Vol. telah dihapus
kolom Change % telah dihapus


In [192]:
df_kurs['Date'] = pd.to_datetime(df_kurs['Date'], format='%m/%d/%Y') #mengubah tipe data 'Date' menjadi datetime

df_kurs['Price'] = pd.to_numeric(df_kurs['Price'].str.replace(',','')) #mengubah 'price' menjadi bentuk numeric

print('data date telah diubah menjadi datetime')
print('data price telah diubah menjadi numeric')

data date telah diubah menjadi datetime
data price telah diubah menjadi numeric


In [193]:
change_name = {
    'Date' : 'date',
    'Price' : 'usdidr'
}

df_kurs = df_kurs.rename(columns=change_name) #ubah nama kolom

for i, j in change_name.items():
    print(f"kolom {i} telah diubah nama menjadi {j}")

kolom Date telah diubah nama menjadi date
kolom Price telah diubah nama menjadi usdidr


In [194]:
df_kurs.head() #melihat sebaran data setelah perubahan nama kolom

,date,usdidr
0,2025-11-19,16716.6
1,2025-11-18,16728.3
2,2025-11-17,16725.0
3,2025-11-14,16694.0
4,2025-11-13,16725.0


## Suku Bunga

In [195]:
df_sb.head() #melihat sebaran data dalam dataframe

,no,tanggal,suku_bunga
0,1,19 November 2025,4.75 %
1,2,22 Oktober 2025,4.75 %
2,3,17 September 2025,4.75 %
3,4,20 Agustus 2025,5.00 %
4,5,16 Juli 2025,5.25 %


In [196]:
df_sb.info() #melihat informasi dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   no          71 non-null     int64 
 1   tanggal     71 non-null     object
 2   suku_bunga  71 non-null     object
dtypes: int64(1), object(2)
memory usage: 1.8+ KB


In [197]:
df_sb = df_sb.drop(columns=('no')) #menghapus kolom 'no' yang tidak digunakan
df_sb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   tanggal     71 non-null     object
 1   suku_bunga  71 non-null     object
dtypes: object(2)
memory usage: 1.2+ KB


In [198]:
df_sb.head() #melihat sebaran data setelah penghapusan kolom 'no'

,tanggal,suku_bunga
0,19 November 2025,4.75 %
1,22 Oktober 2025,4.75 %
2,17 September 2025,4.75 %
3,20 Agustus 2025,5.00 %
4,16 Juli 2025,5.25 %


In [199]:
df_sb['suku_bunga'] = pd.to_numeric(df_sb['suku_bunga'].str.replace(' %', '')) #mengubah kolom suku_bunga menjadi numeric
df_sb.head()

,tanggal,suku_bunga
0,19 November 2025,4.75
1,22 Oktober 2025,4.75
2,17 September 2025,4.75
3,20 Agustus 2025,5.00
4,16 Juli 2025,5.25


In [200]:
#bulan dalam english
bulan = {
    "Januari": "January",
    "Februari": "February",
    "Maret": "March",
    "April": "April",
    "Mei": "May",
    "Juni": "June",
    "Juli": "July",
    "Agustus": "August",
    "September": "September",
    "Oktober": "October",
    "November": "November",
    "Desember": "December"
}

df_sb['tanggal'] = df_sb['tanggal'].replace(bulan, regex=True) #ubah bulan menjadi dalam english
df_sb['tanggal'] = pd.to_datetime(df_sb['tanggal']) #ubah 'tanggal' menjadi datetime
df_sb.head()

,tanggal,suku_bunga
0,2025-11-19,4.75
1,2025-10-22,4.75
2,2025-09-17,4.75
3,2025-08-20,5.00
4,2025-07-16,5.25


In [201]:
df_sb = df_sb.rename(columns={
    'tanggal' : 'date'
}) # ganti nama kolom date menjadi tanggal

df_sb.head(5)

,date,suku_bunga
0,2025-11-19,4.75
1,2025-10-22,4.75
2,2025-09-17,4.75
3,2025-08-20,5.00
4,2025-07-16,5.25


In [202]:
df_sb.info() #melihat informasi dataset setelah perubahan

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        71 non-null     datetime64[ns]
 1   suku_bunga  71 non-null     float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 1.2 KB


In [203]:
df_sb.describe()

,date,suku_bunga
count,71,71.000000
mean,2022-12-19 11:09:17.746478848,4.827465
min,2020-01-23 00:00:00,3.500000
25%,2021-07-04 12:00:00,3.625000
50%,2022-12-22 00:00:00,4.750000
75%,2024-06-05 12:00:00,5.750000
max,2025-11-19 00:00:00,6.250000
std,NaN,1.047609


# Save Data Preprocessing

In [204]:
name = ['df_idx', 'df_nyse', 'df_kurs', 'df_sb'] #name data
data = [df_idx, df_nyse, df_kurs, df_sb] #dataframe

for i, j in zip(name, data): # for untuk menyandingkan indeks yang sama (0,0), (1,1), dst
    path = f'../dataset/preprocessing/{i}_clean.pkl' # buat nama data
    j.to_pickle(path) # save data to pickle
    print(f'dataset {i} telah disimpan di {path}')

dataset df_idx telah disimpan di ../dataset/preprocessing/df_idx_clean.pkl
dataset df_nyse telah disimpan di ../dataset/preprocessing/df_nyse_clean.pkl
dataset df_kurs telah disimpan di ../dataset/preprocessing/df_kurs_clean.pkl
dataset df_sb telah disimpan di ../dataset/preprocessing/df_sb_clean.pkl
